In [50]:
#as it turned out interactive shell (like Jupyter cannot handle CPU multiprocessing well so check which medium the code is runing)
#we will write code in Jupyter for understanding purposes but final execuation will be in shell
from ipynb.fs.full.Utils import isnotebook
from ipynb.fs.full.Dataset import get_data
from torch_geometric.utils import degree
import math
import os

In [51]:
import os.path as osp
from typing import Optional, List, Dict

import torch
from torch_sparse import SparseTensor
from tqdm import tqdm
from torch_geometric.typing import EdgeType, InputNodes

# Testing and Weighted Random Walk

In [52]:
# from torch_geometric.data import Data
# x = torch.Tensor([[1,0],[1,0],[1,0],[0,1],[0,1],[0,1],[0,1]])
# y = torch.LongTensor([0,0,0, 1, 1, 1, 1])
# edge_index = torch.LongTensor([[1,2],[1,4],[1,5],[2,1],[3,6],[3,7],[4,5],[4,1],[4,6],[4,7],[5,1],[5,4],[5,6],[6,3],[6,4],[6,5],[6,7],[7,3],[7,4],[7,6]]).T
# # edge_index = torch.LongTensor([[1,2]]).T


# edge_index = edge_index-1
# data = Data(x=x, y=y, edge_index = edge_index)



# # loader = AGSNodeSampler(data, log = False, batch_size=2, num_steps=4, sample_coverage=1, save_dir='',num_workers=0)

In [53]:
# N = data.num_nodes
# E = data.num_edges

# adj = SparseTensor(
#     row=data.edge_index[0], col=data.edge_index[1],
#     value=torch.arange(E, device=data.edge_index.device),
#     sparse_sizes=(N, N))

# adj

In [54]:
# adj.saint_subgraph(torch.LongTensor([0,4,3]))

In [55]:
import sys
#import torch
#sys.path.append("/home/sferdou/CPPSamplerNew/build/src")

sys.path.append("/home/das90/GNNcodes/CVE2020/GNN-NC/Graph-Sparsification/CPPsamplerPy/build/src")

import sampling_module

In [56]:
# import torch
# from torch_sparse.tensor import SparseTensor
 
def w_random_walk_cpp(src: SparseTensor, start: torch.Tensor, prob: torch.Tensor, walk_length: int) -> torch.Tensor:
    
    rowptr, col, _ = src.csr()    
    return sampling_module.weighted_random_walk(rowptr, col, start, walk_length, prob)

SparseTensor.w_random_walk_cpp = w_random_walk_cpp

In [57]:
def random_walk_cpu(rowptr, col, start, walk_length):
    assert rowptr.device.type == 'cpu'
    assert col.device.type == 'cpu'
    assert start.device.type == 'cpu'

    assert rowptr.dim() == 1
    assert col.dim() == 1
    assert start.dim() == 1

    rand = torch.rand((start.size(0), walk_length), dtype=torch.float32, device=start.device)

    L = walk_length + 1
    out = torch.full((start.size(0), L), -1, dtype=torch.int64, device=start.device)

    for n in range(start.size(0)):
        cur = start[n]
        out[n, 0] = cur

        for l in range(walk_length):
            row_start = rowptr[cur]
            row_end = rowptr[cur + 1]
            
            random_val = rand[n, l].item()
            selected_col_idx = int(random_val * (row_end - row_start))
            cur = col[row_start + selected_col_idx]
            out[n, l + 1] = cur

    return out


In [58]:
def weighted_random_walk_cpu(rowptr, col, start, walk_length, prob):
#     assert rowptr.device.type == 'cpu'
#     assert col.device.type == 'cpu'
#     assert start.device.type == 'cpu'
#     assert prob.device.type == 'cpu'

    assert rowptr.dim() == 1
    assert col.dim() == 1
    assert start.dim() == 1
    assert prob.dim() == 1

    #rand = torch.rand((start.size(0), walk_length), dtype=torch.float32, device=start.device)

    L = walk_length + 1
    out = torch.full((start.size(0), L), -1, dtype=torch.int64, device=start.device)

    for n in range(start.size(0)):
        cur = start[n]
        out[n, 0] = cur

        for l in range(walk_length):
            row_start = rowptr[cur]
            row_end = rowptr[cur + 1]
            
            if int(row_end-row_start)<1:
                out[n, l + 1] = cur
                continue
                
            #random_val = rand[n, l].item()            
            col_probs = prob[row_start:row_end]
            col_probs = torch.clamp(col_probs, min=0.0001)
            col_probs = col_probs/col_probs.sum()
            #print(col_probs)            
            selected_col_idx = torch.multinomial(col_probs, num_samples=1, replacement=True)            
            #selected_col_idx = int(random_val * (row_end - row_start))
            cur = col[row_start + selected_col_idx]
            out[n, l + 1] = cur
    return out


In [59]:
# import torch
# from torch_sparse.tensor import SparseTensor
 
def w_random_walk(src: SparseTensor, start: torch.Tensor, prob: torch.Tensor, walk_length: int) -> torch.Tensor:
    
    rowptr, col, _ = src.csr()    
    return weighted_random_walk_cpu(rowptr, col, start, walk_length, prob)
#     return random_walk_cpu(rowptr, col, start, walk_length)

SparseTensor.w_random_walk = w_random_walk

In [60]:
# #adj.storage.row()[[1,7]]
# deg_in = 1/adj.storage.rowcount()
# print(deg_in)
# deg_out =  1/adj.storage.colcount()
# print(deg_out)
# row, col, _ = adj.coo()

# print(row)
# print(col)
# prob = (1. / deg_in[row]) + (1. / deg_out[col])
# print(prob)


# # prob[1]=0
# # prob[6]=0

# # prob = prob/prob.sum()
# # print(prob)

In [61]:
# start = torch.LongTensor([0])

# for i in range(10):
#     print(adj.w_random_walk_cpp(start, prob, 2))    
# #     print(adj.random_walk(start, 2))

In [62]:
import copy

## Save and load 

In [63]:
import os

def save_weight(method,save_dir,weights):
    filename= save_dir+method+".pt"
    
    directory = osp.dirname(filename)    
    if not osp.exists(directory):
        os.makedirs(directory)
    
    torch.save(weights, filename)
    
def load_weight(method, save_dir):
    filename= save_dir+method+".pt"
    if not osp.exists(filename):
        return None
    else:
        return torch.load(filename)
    
def is_compute(recompute, method, save_dir):
    compute=False
    w = None
    if recompute == True:                         
        compute=True
    else:
        w = load_weight(method, save_dir)
        if w is None:
            compute=True            
    return compute, w

# Graph Sampler

In [64]:
from ipynb.fs.full.DisjointSparsifier import get_random_sparse

In [65]:
class AGSSampler(torch.utils.data.DataLoader):

    def __init__(self, data, batch_size: int, num_steps: int = 1,
                 sample_coverage: int = 0, save_dir: Optional[str] = None, 
                 recompute = True,
                 log: bool = True, 
                 sample_func: List = None,
                 weight_func: Dict = None,
                 params=None,
                 **kwargs):

        if 'collate_fn' in kwargs:
            del kwargs['collate_fn']

        assert data.edge_index is not None
        assert 'node_norm' not in data
        assert 'edge_norm' not in data
        assert not data.edge_index.is_cuda

        self.num_steps = num_steps
        self.__batch_size__ = batch_size
        self.sample_coverage = sample_coverage
        self.save_dir = save_dir
        self.recompute = recompute
        self.log = log

        self.N = N = data.num_nodes
        self.E = data.num_edges

        self.adj = SparseTensor(
            row=data.edge_index[0], col=data.edge_index[1],
            value=torch.arange(self.E, device=data.edge_index.device),
            sparse_sizes=(N, N))

        self.data = data

        super().__init__(self, batch_size=1, collate_fn=self.__collate__,
                         **kwargs)
    
        ### default norm computation
        ### edge_weight computation
        self.sample_func = sample_func
        self.weight_func=weight_func
        self.params=params        
        self.recompute = recompute
        
        self.norm_computation = False        
        self.norm_compute()
        
        if sample_func is not None:            
            weights_dict = {}                    
            for i,method in enumerate(self.sample_func):
                weights_dict[self.weight_func[i]['weight']]=0
                
            #print(weights_dict)
            
            self.computed_weights = {}
            
            for key, value in weights_dict.items():                
                if key == 'knn':
                    w = self.knn_weights() 
                    self.computed_weights[key] = w
                elif key == 'submodular':
                    w = self.submodular_weights() 
                    self.computed_weights[key] = w
                elif key == 'random':
                    w = self.__compute_degree_weight()
                    self.computed_weights[key] = w 
                
                elif key == 'link':
                    w = self.link_weights()
                    self.computed_weights[key] = w 
                
                elif key == 'fastlink':
                    w = self.fastlink_weights(**kwargs)
                    self.computed_weights[key] = w                 
                
                elif key == 'link-nn':
                    w = self.link_nn_weights()
                    self.computed_weights[key] = w
                elif key == 'link-sub':
                    w = self.link_sub_weights()
                    self.computed_weights[key] = w
                    
                elif key == 'apricot':
                    w = self.apricot_sub_weights()
                    self.computed_weights[key] = w
                    
                elif key == 'disjoint':
                    w = self.__compute_degree_weight()
                    self.computed_weights[key] = w
                    self.epoch_count = 0
                    self.copy_data = copy.deepcopy(data)                    
                    self.get_sparse()
    
    def get_sparse(self, sel_K=1):
        sparse_data, sel_mst_index = get_random_sparse(
            self.copy_data, sel_K = sel_K, max_K = self.params['disjoint']['max_K'], metric = self.params['disjoint']['metric'], 
            minimum=self.params['disjoint']['minimum'], draw=False, log=self.log, 
            dataset_name=self.save_dir, recompute=self.recompute)
        
        
        self.sparse_adj = SparseTensor(
            row=sparse_data.edge_index[0], col=sparse_data.edge_index[1],
            value=torch.arange(sparse_data.num_edges, device=sparse_data.edge_index.device),
            sparse_sizes=(sparse_data.num_nodes, sparse_data.num_nodes)
        )
        
        if self.log:
            print(sparse_data, sel_mst_index)
        
        return self.sparse_adj, sel_mst_index   
        
    
    def norm_compute(self):        
        if self.sample_coverage > 0:                
            path = osp.join(self.save_dir or '', self.__filename__)
            directory = osp.dirname(path)    
            if not osp.exists(directory):
                os.makedirs(directory)
            
            if self.recompute == False and self.save_dir is not None and osp.exists(path):  # pragma: no cover
                if self.log:
                    print('loading saved norm')
                
                self.node_norm, self.edge_norm = torch.load(path)
            else:
                self.norm_computation = True
                self.node_norm, self.edge_norm = self.__compute_norm__()
                self.norm_computation = False
                
                if self.log:
                    print('Saving norm')
                
                if self.save_dir is not None:  # pragma: no cover
                    torch.save((self.node_norm, self.edge_norm), path)
            
    def knn_weights(self):        
        from ipynb.fs.full.KNNWeights import KNNWeight

        method = 'knn'
        m_name = method+self.params[method]['metric']
        
        compute, w = is_compute(self.recompute, m_name, self.save_dir)
                        
        if compute:
            knn = KNNWeight(self.data, metric=self.params[method]['metric'], log=self.log)                
            w = knn.compute_weights()
            
            if self.log:
                print("saving weights ",m_name)
            save_weight(m_name, self.save_dir, w)
        else:
            if self.log:
                print("Loading weights ",m_name)
                
        return w
    
    def submodular_weights(self):
        from ipynb.fs.full.SubmodularWeights import SubModularWeightFacilityFaster
        
        method = 'submodular'
        m_name = method+self.params[method]['metric']
        
        compute, w = is_compute(self.recompute, m_name, self.save_dir)
                        
        if compute:
            sub = SubModularWeightFacilityFaster(self.data, metric=self.params[method]['metric'], log=self.log)
            w = sub.compute_weights()
            
            if self.log:
                print("saving weights ",m_name)
            save_weight(m_name, self.save_dir, w)
        else:
            if self.log:
                print("Loading weights ",m_name)
                
        return w
    
    def apricot_sub_weights(self):
        from ipynb.fs.full.SubmodularWeightsApricot import SubModularWeightApricot
        
        method = 'apricot'
        m_name=method+self.params[method]['sub_func']+self.params[method]['metric']
        
        compute, w = is_compute(self.recompute, m_name, self.save_dir)
                        
        if compute:
            sub = SubModularWeightApricot(self.data, metric=self.params[method]['metric'], sub_func=self.params[method]['sub_func'], log=self.log)
            w = sub.compute_weights()
            
            if self.log:
                print("saving weights ",m_name)
            save_weight(m_name, self.save_dir, w)
        else:
            if self.log:
                print("Loading weights ",m_name)
                
        return w
    
    def link_weights(self):
        method = 'link'        
        compute, w = is_compute(self.recompute, method, self.save_dir)
                        
        if compute:
            from ipynb.fs.full.PretrainedLink import LinkPred
            
            linkpred = LinkPred(self.data, selfloop = True, log=self.log)
            w = linkpred.compute_weights()   
            
            if self.log:
                print("saving weights ",method)
            save_weight(method, self.save_dir, w)
        else:
            if self.log:
                print("Loading weights ",method)
                
        return w
    
    def fastlink_weights(self, **kwargs):
        method = 'fastlink'        
        compute, w = is_compute(self.recompute, method, self.save_dir)
                        
        if compute:
            from ipynb.fs.full.PretrainedLinkFast import get_link_weight
            
            w = get_link_weight(self.data, selfloop = True, log = self.log, worker=kwargs['num_workers'])
            
            if self.log:
                print("saving weights ",method)
            save_weight(method, self.save_dir, w)
        else:
            if self.log:
                print("Loading weights ",method)
                
        return w
    
    def link_nn_weights(self):
        method = 'link-nn'        
        compute, w = is_compute(self.recompute, method, self.save_dir)
                        
        if compute:
            #from ipynb.fs.full.PretrainedLink import LinkPred, LinkNN
            from ipynb.fs.full.PretrainedLinkFast import LinkNN
#             default value = 'min'            
#             linkpred = LinkPred(self.data, selfloop = True, log=self.log)
#             self.data.weight = linkpred.compute_weights()   
            
            nn_weight = LinkNN(self.data, value=self.params[method]['value'], log=self.log) #min favor similar ones, max disimilar
            w = nn_weight.compute_weights()                            
            
            if self.log:
                print("saving weights ",method)
            save_weight(method, self.save_dir, w)
        else:
            if self.log:
                print("Loading weights ",method)
                
        return w
    
    def link_sub_weights(self):
        method = 'link-sub'
        compute, w = is_compute(self.recompute, method, self.save_dir)
                        
        if compute:
#             from ipynb.fs.full.PretrainedLink import LinkSub
            from ipynb.fs.full.PretrainedLinkFast import LinkSub
            
            linksub = LinkSub(self.data, value=self.params[method]['value'], selfloop = True, log=self.log) #min favor similar ones, max disimilar    
            w = linksub.compute_weights()                        
                            
            if self.log:
                print("saving weights ",method)
            save_weight(method, self.save_dir, w)
        else:
            if self.log:
                print("Loading weights ",method)
                
        return w
    
    @property
    def __filename__(self):
        return f'{self.__class__.__name__.lower()}_{self.sample_coverage}.pt'

    def __len__(self):
        return self.num_steps

    def __sample_nodes__(self, batch_size):
        raise NotImplementedError

        
    def __getitem__original(self, idx):
        node_idx = self.__sample_nodes__(self.__batch_size__).unique()  
        node_idx = node_idx.clamp(max=self.N-1)
        adj, _ = self.adj.saint_subgraph(node_idx)
        return node_idx, adj
    
    
    def __getitem__(self, idx):        
        if self.norm_computation:
            return self.__getitem__original(idx)
        
#         print("-"*100)        
#         print(idx)    
        
        seed_node, subgraph_node_idx = self.ags_sample_nodes__(idx, self.__batch_size__) #make sure .unique()
        
#         print(seed_node,subgraph_node_idx)
        
        adj_subgraph = []
        for node_idx in subgraph_node_idx:             
            #node_idx = node_idx.clamp(max=self.N-1)            
            adj, _ = self.adj.saint_subgraph(node_idx)            
            adj_subgraph.append(adj)
        
#         print(adj_subgraph)
        
#         print("-"*100)
        
        return seed_node, subgraph_node_idx, adj_subgraph

    def __collate__(self, data_list):
        assert len(data_list) == 1
        
        seed_node, subgraph_node_idx, adj_subgraph = data_list[0]
        
        #node_idx, adj = data_list[0]
        
        batch_data = []
        
        for i in range(len(subgraph_node_idx)):
            
            node_idx = subgraph_node_idx[i]
            adj = adj_subgraph[i]
        
            data = self.data.__class__()
            
            data.seed_node = seed_node
            data.batch_size = seed_node.shape[0]                
            
            data.num_nodes = node_idx.size(0)
            row, col, edge_idx = adj.coo()
            data.edge_index = torch.stack([row, col], dim=0)

            for key, item in self.data:
                if key in ['edge_index', 'num_nodes']:
                    continue
                if isinstance(item, torch.Tensor) and item.size(0) == self.N:
                    data[key] = item[node_idx]
                elif isinstance(item, torch.Tensor) and item.size(0) == self.E:
                    data[key] = item[edge_idx]
                else:
                    data[key] = item
                     
            if self.sample_coverage > 0:
                data.node_norm = self.node_norm[node_idx]
                data.edge_norm = self.edge_norm[edge_idx]

            batch_data.append(data)
            
        if len(batch_data) == 1:
            return batch_data[0]
        
        return batch_data
    
    
    def __compute_degree_weight(self):
        row, col, _ = self.adj.coo()

        deg_in = 1. / self.adj.storage.colcount()
        deg_out = 1. / self.adj.storage.rowcount()
        weight = (1. / deg_in[row]) + (1. / deg_out[col])
        
        return weight
    

    def __compute_norm__(self):
        
        
        node_count = torch.zeros(self.N, dtype=torch.float)
        edge_count = torch.zeros(self.E, dtype=torch.float)
                
        loader = torch.utils.data.DataLoader(self, batch_size=200,
                                             collate_fn=lambda x: x,
                                             num_workers=self.num_workers)

        if self.log:  # pragma: no cover
            pbar = tqdm(total=self.N * self.sample_coverage)
            pbar.set_description('Compute AGS normalization')

        num_samples = total_sampled_nodes = 0
        
        while total_sampled_nodes < self.N * self.sample_coverage:
            
            for data in loader:
                #print(data)                
                for node_idx, adj in data:
                    edge_idx = adj.storage.value()
                    node_count[node_idx] += 1
                    edge_count[edge_idx] += 1
                    total_sampled_nodes += node_idx.size(0)

                    if self.log:  # pragma: no cover
                        pbar.update(node_idx.size(0))
            num_samples += self.num_steps

        if self.log:  # pragma: no cover
            pbar.close()

        row, _, edge_idx = self.adj.coo()
        t = torch.empty_like(edge_count).scatter_(0, edge_idx, node_count[row])
        edge_norm = (t / edge_count).clamp_(0, 1e4)
        edge_norm[torch.isnan(edge_norm)] = 0.1

        node_count[node_count == 0] = 0.1
        node_norm = num_samples / node_count / self.N

        return node_norm, edge_norm


In [66]:
# import torch
# a = torch.LongTensor([1,2,3,4,5])
# a.clamp(max=3)

In [67]:
class AGSNodeSampler(AGSSampler):    
    def __sample_nodes__(self, batch_size):
        edge_sample = torch.randint(0, self.E, (batch_size, self.batch_size),
                                    dtype=torch.long)
        
        print(edge_sample.shape)
        print(edge_sample)
        print(self.adj.storage.row()[edge_sample])

        return self.adj.storage.row()[edge_sample]


In [68]:
class AGSEdgeSampler(AGSSampler):
    
    def __sample_nodes__(self, batch_size):
        row, col, _ = self.adj.coo()

        deg_in = 1. / self.adj.storage.colcount()
        deg_out = 1. / self.adj.storage.rowcount()
        prob = (1. / deg_in[row]) + (1. / deg_out[col])

        # Parallel multinomial sampling (without replacement)
        # https://github.com/pytorch/pytorch/issues/11931#issuecomment-625882503
        rand = torch.rand(batch_size, self.E).log() / (prob + 1e-10)
        edge_sample = rand.topk(self.batch_size, dim=-1).indices

        source_node_sample = col[edge_sample]
        target_node_sample = row[edge_sample]

        return torch.cat([source_node_sample, target_node_sample], -1)
    
    #modify this

In [69]:
class AGSRandomWalkSampler(AGSSampler):
    def __init__(self, data, batch_size: int, walk_length: int,
                 num_steps: int = 1, sample_coverage: int = 0,
                 save_dir: Optional[str] = None, log: bool = True, **kwargs):
        self.walk_length = walk_length
        super().__init__(data, batch_size, num_steps, sample_coverage,
                         save_dir, log, **kwargs)

    @property
    def __filename__(self):
        return (f'{self.__class__.__name__.lower()}_{self.walk_length}_'
                f'{self.sample_coverage}.pt')

    def __sample_nodes__(self, batch_size):
        start = torch.randint(0, self.N, (batch_size, ), dtype=torch.long)
        node_idx = self.adj.random_walk(start.flatten(), self.walk_length)
        return node_idx.view(-1)

In [70]:
class AGSGraphSampler(AGSSampler):
    def __init__(self, data, batch_size: int, walk_length: int =2,
                 num_steps: int = 1, sample_coverage: int = 100,
                 save_dir: Optional[str] = None, log: bool = True, 
                 input_nodes: InputNodes = None,
                 sample_func: List = None,
                 weight_func: Dict = None,
                 params=None,                 
                 recompute = False,                 
                 **kwargs):
        
        self.log = log
        self.sample_func = sample_func
        self.weight_func=weight_func
        self.params=params        
        self.recompute = recompute,
        self.walk_length = walk_length
        self.input_nodes = None
        if 'shuffle' in kwargs:
            self.shfl = kwargs['shuffle']
        else:
            self.shfl = False
        
        if input_nodes is not None:
            self.input_nodes = torch.nonzero(input_nodes).flatten()
            
            
        super().__init__(data, batch_size, num_steps, sample_coverage,
                         save_dir, recompute,log, 
                         sample_func, weight_func,params,
                         **kwargs)
        
    @property
    def __filename__(self):
        return (f'{self.__class__.__name__.lower()}_{self.walk_length}_'
                f'{self.sample_coverage}.pt')

    def ags_sample_nodes__(self, idx, batch_size):                
        
        ## Seed not for everyone
        
        if self.input_nodes is not None:
            if self.shfl:
                rand_node = torch.randint(0, len(self.input_nodes), (batch_size, ), dtype=torch.long).flatten()        
                seed_node = self.input_nodes[rand_node]
            else:                
                seed_node=self.input_nodes[idx*batch_size:(idx+1)*batch_size]
        else:                    
            seed_node = torch.randint(0, self.N, (batch_size, ), dtype=torch.long).flatten()        
        
        seed_unique = seed_node.unique()
        
        #print("start:",seed_node)
        #print("startunique:",seed_unique)
        
        subgraph_node_idx = []   
        
        for i, method in enumerate(self.sample_func):
                                            
            if method in ["rw"]:
                node_idx = self.adj.random_walk(seed_node, self.walk_length)        
                                
            elif method in ['wrw']:          
                weight_name = self.weight_func[i]['weight']
                prob = self.computed_weights[weight_name]                
                #print(prob.shape)                    
                #node_idx = self.adj.w_random_walk(seed_node, prob, self.walk_length)            
                node_idx = self.adj.w_random_walk_cpp(seed_node, prob, self.walk_length)            
            
            elif method in ['disjoint']:                
                node_idx = self.adj.random_walk(seed_node, self.walk_length)     
                
                #print(idx, self.epoch_count)

                if idx == 0 and self.epoch_count > 0:                
                    if self.log:
                        print("Computing sparse graph")                    
                    self.sparse_adj, s_idx = self.get_sparse(self.params['disjoint']['sel_K'])
                    if self.log:
                        print(s_idx)
                    
                    self.epoch_count = self.epoch_count+1
                
                if idx == 0 and self.epoch_count == 0:
                    self.epoch_count+=1
                
                node_idx1 = self.sparse_adj.random_walk(seed_node, self.walk_length)
                node_idx = torch.cat((node_idx,node_idx1))
                
            else:
                raise NotImplementedError
    
            sampled_nodes = node_idx.view(-1)
            
            #print(sampled_nodes)         
            sampled_nodes, inverse_indices = torch.cat((seed_unique, sampled_nodes)).unique(return_inverse=True)            
            n_seed_idx = inverse_indices[:len(seed_unique)]
            to_swap = sampled_nodes[:len(seed_unique)]

            #print(n_seed_idx)
            #print(to_swap)
            #Use scatter to replace elements efficiently
            #sampled_nodes.scatter_(0, n_seed_idx, to_swap.clone())
            #sampled_nodes[:len(seed_node)] = seed_node

            sampled_nodes[n_seed_idx]=to_swap.clone()
            sampled_nodes[:len(seed_unique)]=seed_unique
            
            ## make exact batch size
            if self.weight_func[i]['exact']:
                if len(sampled_nodes)>=batch_size:                
                    sampled_nodes = sampled_nodes[:batch_size]
                else:
                    requirement = batch_size-len(sampled_nodes)
                    #print("node required: ", requirement)
                    mask = torch.ones(self.N, dtype=torch.bool)
                    mask[sampled_nodes]=False
                    other_nodes = torch.nonzero(mask).flatten()

                    #print(other_nodes, len(other_nodes))

                    random_indices = torch.randperm(other_nodes.size(0))[:requirement]
                    extra_sampled = other_nodes[random_indices]
                    sampled_nodes = torch.cat((sampled_nodes,extra_sampled))
                    
                    #print(sampled_nodes)
                assert len(sampled_nodes) == batch_size                                
        
            subgraph_node_idx.append(sampled_nodes)                
            
        #print(seed_unique, subgraph_node_idx)        
        return seed_unique, subgraph_node_idx
    
    def __sample_nodes__(self, batch_size):                
        start = torch.randint(0, self.N, (batch_size, ), dtype=torch.long)
        node_idx = self.adj.random_walk(start.flatten(), self.walk_length)        
        return node_idx.view(-1)

In [71]:
if __name__ == '__main__':  
    
    whole_data, dataset = get_data('karate')        
#     whole_data, dataset = get_data('karate')        
    
#     #file_path = 'graphsaintrandomwalksampler_2_100.pt'
#     #file_path = 'graphsaintrandomwalksampler_100.pt'
#     file_path = 'graphsaintnodesampler_10.pt'
    
#     if os.path.exists(file_path):
#         !rm 'graphsaintnodesampler_10.pt'        
#         print("Deleted: ",file_path)
#     else:
#         print('File not exist')
            
    #sample_func=['rw', 'exact', 'wrw', 'wexact', 'disjoint']
    
#     sample_func =['disjoint']
#     weight_func =[
#         {'exact':False,'weight':'disjoint'}, #exact for exact size to the batch
#         #{'exact':False,'weight':'random'}, #exact for exact size to the batch
#     ]

    sample_func =['wrw','wrw','wrw','wrw','wrw','wrw']
    weight_func =[
        {'exact':False,'weight':'knn'},
        {'exact':False,'weight':'submodular'},
        {'exact':False,'weight':'fastlink'}, 
        {'exact':False,'weight':'link-nn'},
        {'exact':False,'weight':'link-sub'},
        {'exact':False,'weight':'apricot'},
    ]

    ##minimum corresponds to decreasing subgraph
    
    params={'knn':{'metric':'cosine'},
            'submodular':{'metric':'cosine'},
            'link-nn':{'value':'min'},
            'link-sub':{'value':'max'},
            'disjoint':{'value':'mst', 'sel_K':2, 'max_K':5, 'metric':'cosine','minimum':True}, 
            'apricot':{'sub_func':'facility','metric':'cosine'},
           }
    
    batch_size = 2
    num_steps = int(math.ceil(whole_data.num_nodes/batch_size))
    print(num_steps)

    loader = AGSGraphSampler(
        whole_data, batch_size=batch_size, 
        input_nodes = None,
        #input_nodes = whole_data.train_mask, 
        walk_length=2,
        num_steps=num_steps, 
        sample_coverage=2,
        num_workers=0,
        sample_func = sample_func,
        weight_func=weight_func,
        params=params,
        save_dir = 'Results/GS',
        recompute = False,
        shuffle = False,
        log = True,
    )

    batch  = next(iter(loader))
    
    for b in batch:
        print(b)
    
    for i in range(10):
        for batch in loader:
            #print(batch)
            None
    
    None

Data directory:  /scratch/gilbreth/das90/Dataset/
Result directory: /scratch/gilbreth/das90/Dataset/RESULTS/

Dataset: KarateClub():
Number of graphs: 1
Number of features: 34
Number of classes: 4

Data(x=[34, 34], edge_index=[2, 156], y=[34], train_mask=[34], val_mask=[34], test_mask=[34])
Number of nodes: 34
Number of edges: 156
Average node degree: 4.59
Number of training nodes: 4
Training node label rate: 0.12
Has isolated nodes: False
Has self-loops: False
Is undirected: True
17
loading saved norm
Loading weights  knncosine
Loading weights  submodularcosine
Loading weights  fastlink
Loading weights  link-nn
Loading weights  link-sub
Loading weights  apricotfacilitycosine
Data(seed_node=[2], batch_size=2, num_nodes=5, edge_index=[2, 10], x=[5, 34], y=[5], train_mask=[5], val_mask=[5], test_mask=[5], node_norm=[5], edge_norm=[10])
Data(seed_node=[2], batch_size=2, num_nodes=4, edge_index=[2, 6], x=[4, 34], y=[4], train_mask=[4], val_mask=[4], test_mask=[4], node_norm=[4], edge_norm=

In [72]:
# import torch
# a = torch.Tensor([12,23])
# a.clamp(min=1)